In [2]:
import pandas as pd

In [1]:
import os
import sys
# including modules
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import datetime 
import operator
import numpy as np

In [3]:
fn = 'ACS.csv'

#fn = 'ACS_sm.csv'
#dataset = pd.read_csv(filename)
#dataset.head(3)
fn = pd.read_csv(fn)

fn.head(3)


FileNotFoundError: [Errno 2] No such file or directory: 'cpi99'

In [4]:
cpi99='cpi99.csv'
cpi99 = pd.read_csv(cpi99)

In [12]:
#wm = lambda x: np.average(x, weights=dataset.loc[x.index, "perwt"])

In [5]:
fn=pd.merge(fn,cpi99, how='left' ,on=["year"])

In [6]:
fn['ftotinc'].mask( fn['ftotinc'] == 9999999, np.NAN, inplace=True)
fn['ftotinc'].mask( fn['ftotinc'] == 9999998, np.NaN, inplace=True)
fn['ftotinc'].mask( fn['ftotinc'] <0, 0, inplace=True)


In [7]:
fn['ftotinc']=fn['ftotinc']*fn['cpi99']

In [8]:
fn['inctot'].mask( fn['inctot'] == 9999999, np.NAN, inplace=True)
fn['inctot'].mask( fn['inctot'] <0, 0, inplace=True)

In [9]:
fn['inctot']=fn['inctot']*fn['cpi99']

In [10]:
fn['valueh'].mask( fn['valueh'] == 9999999, np.NAN, inplace=True)
fn['valueh'].mask( fn['valueh'] == 9999998, np.NAN, inplace=True)
fn['valueh'].mask( fn['valueh'] <0, 0, inplace=True)

In [11]:
fn['valueh']=fn['valueh']*fn['cpi99']

In [12]:
fn['mortamt1'].mask( fn['mortamt1'] == 0, np.NAN, inplace=True)
fn['mortamt1'].mask( fn['mortamt1'] == 1, np.NAN, inplace=True)
fn['mortamt1'].mask( fn['mortamt1'] <0, 0, inplace=True)

In [13]:
fn['mortamt1']=fn['mortamt1']*fn['cpi99']

In [14]:
fn['foodstmp'].mask( fn['foodstmp'] == 0, np.NAN, inplace=True)

In [15]:
#identifying citizenship status
#by default set everyone as citizen
fn['ctz_stat']=1
#replace naturalized citizens as 2
fn.loc[(fn['citizen'] ==2), 'ctz_stat'] = 2
#replace non citizens as 3
fn.loc[(fn['citizen'] ==3), 'ctz_stat'] = 3
fn.loc[(fn['citizen'] ==4), 'ctz_stat'] = 3



In [16]:
#identifying language
#by default set everyone as speaking English at home
fn['lang']=1
#replace all other languages as 2
fn.loc[((fn['language'] > 1) &(fn['language']<96)), 'lang'] = 2


In [12]:
#This did not work as :Speakeng variable was not found in the dataframe.
#identifying those who can not speak English
#by default set everyone who can speak English 
#fn['eng_speak']=1
#replace those who can not speak English as 2
#fn.loc[(fn['SPEAKENG'] ==3), 'eng_speak'] = 2

In [17]:
#classifying education
#by default set everyone as having a Doctoral Degree as 4
fn['educ_attain']=4
#replace those who have high school diploma or below as 1
fn.loc[(fn['educd'] <65), 'educ_attain'] = 1
#replace those who have some college or bachelor degree as 2
fn.loc[((fn['educd'] >64) &(fn['educd']<102)), 'educ_attain'] = 2
#replace those who have more college education, masters, or a professional degree as 3
fn.loc[((fn['educd'] >102) &(fn['educd']<116)), 'educ_attain'] = 3
# if the value is missing, set it as none
fn.loc[(fn['educd'] ==999), 'educ_attain'] = None

In [18]:
# identfying employment status - setting missing values as missing
fn['empstat'].mask( fn['empstat'] == 0, np.NAN, inplace=True)

In [17]:
# This variable vetstat was not found
#identifying those who are Veterans
#by default set everyone as not Veteran
fn['veteran_stat']=1
#replace those who are veterans
fn.loc[(fn['vetstat'] ==2), 'veteran_stat'] = 2

In [27]:
fn00_04=fn[fn.year<2005]

In [28]:
fn17_20.to_csv('acs17_20.csv')

In [16]:
#dataset['wfamtot']=dataset['ftotinc']*dataset['perwt']

In [17]:
#dataset['all_age']=dataset['age']*dataset['perwt']

In [18]:
#df=dataset.groupby(["year", "state_county"]).agg(All_fam_inc=("wfamtot", "sum"),All_age=("all_age", "sum"),All_weight=("perwt", "sum"),W_avrg_age=("age", wm),Family_income=("ftotinc", wm),Fam_Inc_Avrg=("ftotinc", "mean"), average_mortgage=("mortgage", wm))

In [19]:
#df = pd.DataFrame(df.to_records())

In [20]:
#df=df.round(1)

In [21]:
#df['weighted_fam_inc']=df['All_fam_inc']/df['All_weight']

In [22]:
#df['weighted_age']=df['All_age']/df['All_weight']

In [23]:
#df=df.round(1)

In [24]:
#df.head(3)

,year,state_county,All_fam_inc,All_age,All_weight,W_avrg_age,Family_income,Fam_Inc_Avrg,average_mortgage,weighted_fam_inc,weighted_age
0,2000,1,154835552108,146803569,3191816,46.0,48510.2,51655.0,1.7,48510.2,46.0
1,2000,2,27348275500,17261529,406721,42.4,67240.9,69465.2,1.6,67240.9,42.4
2,2000,4,198811659574,164713718,3639273,45.3,54629.5,58403.5,1.7,54629.5,45.3


In [50]:
def w_avrg(df,data_col,weight_col,by_col):
    ''' Now data_col can be a list of variables '''
    df_data = df[data_col].multiply(df[weight_col], axis='index')
    df_weight = pd.notnull(df[data_col]).multiply(df[weight_col], axis='index')
    df_data[by_col] = df[by_col]
    df_weight[by_col] = df[by_col]    
    result = df_data.groupby(by_col).sum() / df_weight.groupby(by_col).sum()
    result = pd.DataFrame(result.to_records())
    result=result.round(1)
    return result

In [108]:
grp_by=list()

In [109]:
grp_by.append('year')

In [110]:
grp_by.append('state_county')

In [111]:
var_col=list()

In [112]:
var_col.append('age')
var_col.append('ftotinc')

In [113]:
test=w_avrg(fn13_16,var_col,'perwt',grp_by)

In [114]:
test.head(3)

,year,state_county,age,ftotinc
0,2013,1000,47.4,43989.5
1,2013,1003,49.4,47498.9
2,2013,1015,47.2,37874.8


In [115]:
myun=fn13_16.groupby(['year','state_county'],as_index=False).agg({'inctot': 'mean','mortamt1': 'mean', 'sex':'count' })

In [116]:
myun.head(3)

,year,state_county,inctot,mortamt1,sex
0,2013,1000,21147.993858,567.128177,25631
1,2013,1003,25008.576363,696.433539,1071
2,2013,1015,17909.555904,488.263423,785


In [117]:
final=pd.merge(myun,test, how='left' ,on=["year","state_county"])

In [118]:
final.head(3)

,year,state_county,inctot,mortamt1,sex,age,ftotinc
0,2013,1000,21147.993858,567.128177,25631,47.4,43989.5
1,2013,1003,25008.576363,696.433539,1071,49.4,47498.9
2,2013,1015,17909.555904,488.263423,785,47.2,37874.8


In [ ]:
# This code will return percentages of unique values in select columns listed in cols list. IT will create a new column for each unique value in each column

In [31]:
from pandas._libs.lib import maybe_booleans_to_slice

In [119]:
cols=list()

In [33]:
#cols.extend(('foodstmp','sex','marst','race','ctz_stat','lang','eng_speak','educ_attain','empstat','veteran_stat'))

In [120]:
cols.extend(('foodstmp','sex','marst','race','ctz_stat','lang','educ_attain','empstat'))

In [121]:
for column in cols:
  #get the list of distinct answers (but not missing values (NaN)) of the variable in an array -> both myarray and distval are the same at this stage
  #copy the column(variable) into a new temporary one-column dataframe
  tempcol=fn13_16[column]
  tempcol1=tempcol[tempcol!='nan']
  #drop NaN values and get remaining distinct answers
  myarray=tempcol1.dropna().unique().astype(str)
  distval=tempcol1.dropna().unique().astype(str)
  #find number of distinct answers of the question
  mylen=len(myarray)
  # Add the column name (question number) to these distinct answers 
  # at this stage myarray will include the list variable_name+distinct answer that I will use for renaming columns
  for i in range (mylen):
    myarray[i]=column+ '_'+ myarray[i]
  # finding statistics -> grouping by year and state_county and counting the number of each distinct answer
  # Crosstab counts distinct answers but not NaN values. Therefore we do not need to worry about missing values here as they are not counted. 
  # Crosstab produces a table, not a data frame
  test1=pd.crosstab([fn13_16.year, fn13_16.state_county], fn13_16[column])
  # converting the summary statistics table to a dataframe
  flattened = pd.DataFrame(test1.to_records())
  # Renaming the columns to show the variable_name+distinct answer
  for i in range (mylen):
    flattened.rename(columns={distval[i]:myarray[i]}, inplace=True)
  # Adding the counts of each value into a total column
  flattened[column+'_All'] = flattened[myarray].sum(axis=1).astype(int)
  # creating new columns (variable_name+distinct_value+freq) to show the percentages in addition to the counts 
  for i in range (mylen):
    flattened[myarray[i]+'_freq']=(flattened[myarray[i]]/flattened[column+"_All"])*100
  #Rounding all percentages into a single decimal format
  flattened1=np.round(flattened, decimals = 1)
  #flattened=flattened.round(1)
  # add the statistics of the variable to the final dataframe by merging based on key fields.
  final=pd.merge(final,flattened1, how='left' ,on=["year","state_county"])
# convert all missing cells into 0.
final.fillna(0, inplace=True)

In [122]:
final.head(3)

,year,state_county,inctot,mortamt1,sex,age,ftotinc,foodstmp_1,foodstmp_2,foodstmp_All,...,educ_attain_1.0_freq,educ_attain_4.0_freq,educ_attain_3.0_freq,empstat_1.0,empstat_2.0,empstat_3.0,empstat_All,empstat_3.0_freq,empstat_1.0_freq,empstat_2.0_freq
0,2013,1000,21147.993858,567.128177,25631,47.4,43989.5,21870,3761,25631,...,49.3,0.7,6.4,12617,1218,11796,25631,46.0,49.2,4.8
1,2013,1003,25008.576363,696.433539,1071,49.4,47498.9,968,103,1071,...,38.8,0.8,7.7,564,40,467,1071,43.6,52.7,3.7
2,2013,1015,17909.555904,488.263423,785,47.2,37874.8,652,133,785,...,50.3,0.6,5.9,353,55,377,785,48.0,45.0,7.0


In [124]:
final = final.dropna()

In [125]:
final.shape

(1912, 77)

In [126]:
final.to_csv('final13_16.csv')

In [127]:
df1='final00_04.csv'
df1 = pd.read_csv(df1)

In [128]:
df2='final05_08.csv'
df2 = pd.read_csv(df2)

In [129]:
df3='final09_12.csv'
df3 = pd.read_csv(df3)

In [130]:
df4='final13_16.csv'
df4 = pd.read_csv(df4)

In [131]:
df5='final17_20.csv'
df5 = pd.read_csv(df5)

In [132]:
final_data=pd.concat([df1,df2,df3,df4,df5],ignore_index=True)

In [133]:
final_data.shape

(7526, 78)

In [134]:
final_data=np.round(final_data, decimals = 1)

In [135]:
final_data.to_csv('final_data.csv')

In [138]:
election='election.csv'
election = pd.read_csv(election)

In [139]:
data_combined=pd.merge(final_data,election, how='left' ,on=["year","state_county"])

In [140]:
data_combined.to_csv('data_combined.csv')